We focus on books category first; We want to limit the data range to 2 years period; To do so, we want to filter by first review dates of the products in the books category.

https://cseweb.ucsd.edu/~jmcauley/datasets/amazon_v2/

In [2]:
import importlib
import get_initial_dates
importlib.reload(get_initial_dates)
from get_initial_dates import *

/Users/leahtan/Documents/3_Research/2024-Ali/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
data_path = '../../data/'
category = 'Books'
json_file = data_path + category + '.json'

# Read first 3 lines
with open(json_file, 'r') as f:
    for i in range(3):
        print(f'Line {i+1}: {f.readline().strip()}')

Line 1: {"overall": 5.0, "verified": false, "reviewTime": "08 12, 2005", "reviewerID": "A1C6M8LCIX4M6M", "asin": "0001713353", "style": {"Format:": " Paperback"}, "reviewerName": "June Bug", "reviewText": "This book is a winner with both of my boys.  They really enjoy the pictures and the story.  It's a classic.", "summary": "Children's favorite", "unixReviewTime": 1123804800}
Line 2: {"overall": 5.0, "verified": false, "reviewTime": "03 30, 2005", "reviewerID": "A1REUF3A1YCPHM", "asin": "0001713353", "style": {"Format:": " Hardcover"}, "reviewerName": "TW Ervin II", "reviewText": "The King, the Mice and the Cheese by Nancy Gurney is an excellent children's book.  It is one that I well remember from my own childhood and purchased for my daughter who loves it.\n\nIt is about a king who has trouble with rude mice eating his cheese. He consults his wise men and they suggest cats to chase away the mice. The cats become a nuisance, so the wise men recommend the king bring in dogs to chase t

In [13]:
import pandas as pd

# Test data
test_data = {"reviewTime": ["08 12, 2005", "09 15, 2023", "12 31, 2022"]}
df_test = pd.DataFrame(test_data)

# Convert using the correct format
df_test['timestamp'] = pd.to_datetime(df_test['reviewTime'])
print(df_test)

    reviewTime  timestamp
0  08 12, 2005 2005-08-12
1  09 15, 2023 2023-09-15
2  12 31, 2022 2022-12-31


In [17]:
#get the initial dates
category = 'Books'
data_path = '../../data/'
category = 'Books'
json_file = data_path + category + '.json'
first_review_dates = get_first_review_dates_local(json_file, category=category, timestamp_col='reviewTime', version=2018)

Records Processed: 51,310,000
Unique Products: 2,929,846
Memory Usage: 1286.3 MB
Time Elapsed: 2301.8 seconds
Processing Rate: 22291.5 records/second


In [20]:
#test output
path = '../../data/Books/first_review_dates.parquet'
first_review_date = pd.read_parquet(path)
#distribution of first review dates on year basis
first_review_date['year'] = first_review_date['first_review_date'].dt.year
print(first_review_date['year'].value_counts().sort_index())
del first_review_date

In [25]:
#now, filter based on dates; get reviews from 2016 onwards
path = '../../data/Books/first_review_dates.parquet'
data_path = '../../data/'
category = 'Books'
json_file = data_path + category + '.json'
df_first_reviews = pd.read_parquet(path)
recent_asins = set(df_first_reviews[df_first_reviews['first_review_date'] > '2016-01-01'].index)
filtered_data = process_filtered_dataset_local(file_path = json_file, category=category, asins_to_keep=recent_asins, timestamp_col='reviewTime', version=2018) #batch_size=10000, saved with chuncks

Records Processed: 51,300,000
Filtered Records: 76,571
Memory Usage: 1285.8 MB
Processing Rate: 19802.3 records/second
Time Elapsed: 2590.6 seconds
Final chunks saved with 3155 records


In [27]:
#next, get 2023 data of the same asin list
## filter asins using the same var: recent_asins
# Usage with optimizations
category = "Books"
filtered_data = process_filtered_dataset(category=category, asins_to_keep=recent_asins) #2023 data;


Records Processed: 29,400,000
Filtered Records: 8,644
Memory Usage: 1396.8 MB
Processing Rate: 6374.8 records/second
Time Elapsed: 4611.9 seconds


NameError: name 'category_23' is not defined

In [29]:
path1 = '/Users/leahtan/Documents/3_Research/2024-Ali/data/Books/filtered_data_100000.parquet'
path2 = '/Users/leahtan/Documents/3_Research/2024-Ali/data/Books_23/filtered_data_100000.parquet'
display(pd.read_parquet(path1).head())
display(pd.read_parquet(path2).head())

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
8021,5.0,True,2018-04-14,A2R9E427N3BOV2,0001939777,None,Theresa M.,"Everything was greatthe story, the lion (Aslan...",We want six stars!,1523664000,None,None
8022,2.0,True,2018-04-14,A31HTC55E2C7Z,0001939777,None,Ryan O,This is an 8.5x11 (magazine size) paperback co...,Not the C.S. Lewis version,1523664000,None,None
8023,5.0,False,2018-04-09,AL66F5NF82A7N,0001939777,None,BAKER,A classic. I will read this to my kid/grandkid...,Five Stars,1523232000,None,None
8024,5.0,True,2018-04-08,A962F1LM3YN21,0001939777,None,Amazon Customer,Still a great book! I cant wait to read the s...,Classic,1523145600,None,None
8025,5.0,True,2018-04-03,ABR2ZKZ1M1HRM,0001939777,None,Amazon Customer,I wanted to introduce my little cousins to som...,... to introduce my little cousins to some of ...,1522713600,None,None


,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
27,5.0,yep. need this.,yep. need this.,[],0718032152,0718032152,AGCI7FAH4GL5FI65HYLKWTMFZ2CQ,2017-02-22 17:49:51.000,0,False
34,5.0,Five Stars,I love it.,[],0931722470,0931722470,AGKHLEW2SOWHNMFQIJGBECAF7INQ,2017-09-08 00:21:28.659,0,True
35,5.0,Five Stars,Love it.,[],0931722306,0931722306,AGKHLEW2SOWHNMFQIJGBECAF7INQ,2017-09-08 00:21:15.806,0,True
37,5.0,A wonderful start to an interesting world,This is a great start to a world filled with r...,[],1717827136,1717827136,AFTC6ZR5IKNRDG5JCPVNVMU3XV2Q,2018-07-26 20:21:31.287,1,True
48,5.0,As a woman in this industry...,I laughed and cried. Sad that the industry has...,[],1501110624,1501110624,AGBUJDDLRJIUJFTKPABJT6CJTHRQ,2016-02-26 00:16:29.000,0,True


In [ ]:
#may need to add the last batch result

In [48]:
def find_unique_records_in_a(folder_a, folder_b):
    start_time = time.time()
    files_a = list(Path(folder_a).glob('filtered_data_*.parquet'))
    files_b = list(Path(folder_b).glob('filtered_data_*.parquet'))
    
    key_columns_2018 = ['reviewTime', 'reviewerID', 'asin','reviewText']
    key_columns_2023 = ['timestamp', 'user_id', 'asin','text'] 
    
    # Load or create 2023 keys
    save_path = os.path.join(folder_b, 'keys_2023.parquet')
    if os.path.exists(save_path):
        print("Loading 2023 keys...")
        keys_2023_df = pd.read_parquet(save_path)
        keys_2023_basic = set(zip(keys_2023_df['Timekey'], keys_2023_df['user_id'], keys_2023_df['asin']))
        keys_2023_text = set(zip(keys_2023_df['Timekey'], keys_2023_df['asin'], keys_2023_df['text']))
    else:
        print("Building 2023 keys set...")
        keys_all = set()
        for file_b in files_b:
            chunk_b = pd.read_parquet(file_b, columns=key_columns_2023)
            chunk_b['Timekey'] = pd.to_datetime(chunk_b['timestamp'], unit='s').dt.strftime('%Y-%m-%d')
            
            # Create both types of keys
            keys_all.update(zip(chunk_b['Timekey'], chunk_b['user_id'], chunk_b['asin'], chunk_b['text']))
        
        keys_2023_df = pd.DataFrame(keys_all, columns=['Timekey', 'user_id', 'asin', 'text'])
        keys_2023_basic = set(zip(keys_2023_df['Timekey'], keys_2023_df['user_id'], keys_2023_df['asin']))
        keys_2023_text = set(zip(keys_2023_df['Timekey'], keys_2023_df['asin'], keys_2023_df['text']))
        keys_2023_df.to_parquet(save_path)
        print("Keys saved.")
        del keys_2023_df
        del keys_all
    
    # Load or create 2018 keys
    save_path = os.path.join(folder_a, 'keys_2018.parquet')
    if os.path.exists(save_path):
        print("Loading 2018 keys...")
        keys_2018_df = pd.read_parquet(save_path)
        keys_2018_basic = set(zip(keys_2018_df['Timekey'], keys_2018_df['reviewerID'], keys_2018_df['asin']))
        keys_2018_text = set(zip(keys_2018_df['Timekey'], keys_2018_df['asin'], keys_2018_df['reviewText']))
    else:
        print("Building 2018 keys set...")
        keys_2018_all = set()

        for file_a in files_a:
            chunk_a = pd.read_parquet(file_a, columns=key_columns_2018)
            chunk_a['Timekey'] = pd.to_datetime(chunk_a['reviewTime']).dt.strftime('%Y-%m-%d')
            
            keys_2018_all.update(zip(chunk_a['Timekey'], chunk_a['reviewerID'], chunk_a['asin'], chunk_a['reviewText']))
            #form basic and text keys
    
        keys_2018_df = pd.DataFrame(keys_2018_all, columns=['Timekey', 'reviewerID', 'asin', 'reviewText'])
        keys_2018_basic = set(zip(keys_2018_df['Timekey'], keys_2018_df['reviewerID'], keys_2018_df['asin']))
        keys_2018_text = set(zip(keys_2018_df['Timekey'], keys_2018_df['asin'], keys_2018_df['reviewText']))
        keys_2018_df.to_parquet(save_path)
        print("Keys saved.")
        del keys_2018_df
        del keys_2018_all

    # Compare keys
    total18 = len(keys_2018_basic)
    print("\nComparing keys...")
    print(f"1) Basic key comparison (Timekey, ID, ASIN):")
    unique_basic = keys_2018_basic - keys_2023_basic
    print(f"Records unique to 2018: {len(unique_basic):,}")
    print(f"Proportion unique to 2018: {len(unique_basic)/total18:.2%}")
    
    print(f"\n2) Text comparison:")
    unique_text = keys_2018_text - keys_2023_text
    print(f"Reviews with unique text: {len(unique_text):,}")
    print(f"Proportion unique text: {len(unique_text)/total18:.2%}")
    
    return {
        'unique_basic_keys': unique_basic,
        'unique_text': unique_text
    }

In [49]:
folder_a = '/Users/leahtan/Documents/3_Research/2024-Ali/data/Books'
folder_b = '/Users/leahtan/Documents/3_Research/2024-Ali/data/Books_23'
unique_records_18 = find_unique_records_in_a(folder_a, folder_b)

Building 2023 keys set...
Keys saved.
Building 2018 keys set...
Keys saved.

Comparing keys...
1) Basic key comparison (Timekey, ID, ASIN):
Records unique to 2018: 7,764,565
Proportion unique to 2018: 100.00%

2) Text comparison:
Reviews with unique text: 6,912,054
Proportion unique text: 89.02%


In [8]:
#we should focus on asins which are in both 2018 and 2023 data; 
# some products may be deleted completely in 2023;
# check asins in both 2018 and 2023 data

def compare_keys(keys_2018_df, keys_2023_df):
    start_time = time.time()
    
    # Get common ASINs
    asins_2018 = set(keys_2018_df['asin'])
    asins_2023 = set(keys_2023_df['asin'])
    common_asins = asins_2018.intersection(asins_2023)
    print(f"Common asins: {len(common_asins):,}")
    print(f"Unique asins in 2018: {len(asins_2018 - asins_2023):,}")
    print(f"Unique asins in 2023: {len(asins_2023 - asins_2018):,}")
    print(f'proportion of common asins: {len(common_asins)/len(asins_2018):.2%}')
    
    # Filter dataframes to only include common ASINs
    keys_2018_df = keys_2018_df[keys_2018_df['asin'].isin(common_asins)]
    keys_2023_df = keys_2023_df[keys_2023_df['asin'].isin(common_asins)]
    
    # Create key sets for comparison
    keys_2018_basic = set(zip(keys_2018_df['Timekey'], keys_2018_df['reviewerID'], keys_2018_df['asin']))
    keys_2018_text = set(zip(keys_2018_df['Timekey'], keys_2018_df['asin'], keys_2018_df['reviewText']))
    
    keys_2023_basic = set(zip(keys_2023_df['Timekey'], keys_2023_df['user_id'], keys_2023_df['asin']))
    keys_2023_text = set(zip(keys_2023_df['Timekey'], keys_2023_df['asin'], keys_2023_df['text']))
    
    # Compare keys
    total18 = len(keys_2018_basic)
    print("\nComparing keys (only for common ASINs)...")
    print(f"1) Basic key comparison (Timekey, ID, ASIN):")
    unique_basic = keys_2018_basic - keys_2023_basic
    print(f"Records unique to 2018: {len(unique_basic):,}")
    print(f"Proportion unique to 2018: {len(unique_basic)/total18:.2%}")
    
    print(f"\n2) Text comparison:")
    unique_text = keys_2018_text - keys_2023_text
    print(f"Reviews with unique text: {len(unique_text):,}")
    print(f"Proportion unique text: {len(unique_text)/total18:.2%}")



In [ ]:
#compare common keys within common asins
#also, remove asins if all its reviews are unique to 2018;
keys_2018_df = pd.read_parquet('/Users/leahtan/Documents/3_Research/2024-Ali/data/Books/keys_2018.parquet')
keys_2023_df = pd.read_parquet('/Users/leahtan/Documents/3_Research/2024-Ali/data/Books_23/keys_2023.parquet')

In [7]:
compare_keys(keys_2018_df, keys_2023_df)

Common asins: 451,523
Unique asins in 2018: 168,003
Unique asins in 2023: 0
proportion of common asins: 72.88%

Comparing keys (only for common ASINs)...
1) Basic key comparison (Timekey, ID, ASIN):
Records unique to 2018: 6,930,715
Proportion unique to 2018: 100.00%

2) Text comparison:
Reviews with unique text: 6,078,163
Proportion unique text: 87.70%


{'unique_basic_keys': {('2016-10-27', 'ABCQL4PO0YPSN', '0778319482'),
  ('2018-01-11', 'A1OK2AUZLP7ZJQ', '0998063606'),
  ('2017-11-13', 'A2JUCCG7A83GER', '1503942643'),
  ('2017-09-03', 'AD9RC8R8G3E01', '1534659021'),
  ('2017-02-15', 'AUINFADB23VKQ', '1929214111'),
  ('2017-10-12', 'A3TIJ9R7W5RAFW', '1683247353'),
  ('2018-02-14', 'A3FCAOKNFJEFRU', '1947392123'),
  ('2016-09-24', 'AJ7GM42B6IRE7', '1537176269'),
  ('2017-01-26', 'A8090ZPBH1YTV', '054401846X'),
  ('2016-03-24', 'A3LT87TAHQN0R7', '0996143017'),
  ('2018-04-16', 'A1BXJ6JF5BAHWU', '1980792011'),
  ('2016-07-09', 'A2RA513MA8GR5W', '1910751774'),
  ('2017-09-22', 'AETAIOWSW5TKU', '1633220532'),
  ('2017-11-30', 'AOSTZRBDSDDFD', '1975631889'),
  ('2017-02-09', 'A3Q0SJGX4FBA3J', '099665769X'),
  ('2017-10-09', 'A1RDJW5EBA4TT3', '1945796324'),
  ('2017-08-01', 'A1HHSUAQDZ1XM6', '152104337X'),
  ('2016-12-02', 'A2V9FUX86IE2ZH', '1945932015'),
  ('2017-12-07', 'A39ILQVLROUXYN', '0692962557'),
  ('2017-09-07', 'A3CB88FAA72BTC', '